In [4]:
import os
import numpy as np
import rasterio
import matplotlib.image
import matplotlib.pyplot as plt

from pathlib import Path


In [5]:
files = {}

for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        if parts[1][:6] == "202204":
            if parts[0] not in files:
                files[parts[0]] = {}
            if parts[1] not in files[parts[0]]:
                files[parts[0]][parts[1]] = {}
            files[parts[0]][parts[1]][parts[2]] = f"data/{f}"


In [6]:
files


{'T34UDG': {'20220422T095031': {'B02': 'data/T34UDG_20220422T095031_B02_10m.jp2',
   'B03': 'data/T34UDG_20220422T095031_B03_10m.jp2',
   'B04': 'data/T34UDG_20220422T095031_B04_10m.jp2',
   'B08': 'data/T34UDG_20220422T095031_B08_10m.jp2',
   'SCL': 'data/T34UDG_20220422T095031_SCL_20m.jp2'},
  '20220412T095031': {'B02': 'data/T34UDG_20220412T095031_B02_10m.jp2',
   'B03': 'data/T34UDG_20220412T095031_B03_10m.jp2',
   'B04': 'data/T34UDG_20220412T095031_B04_10m.jp2',
   'B08': 'data/T34UDG_20220412T095031_B08_10m.jp2',
   'SCL': 'data/T34UDG_20220412T095031_SCL_20m.jp2'},
  '20220417T095029': {'B02': 'data/T34UDG_20220417T095029_B02_10m.jp2',
   'B03': 'data/T34UDG_20220417T095029_B03_10m.jp2',
   'B04': 'data/T34UDG_20220417T095029_B04_10m.jp2',
   'B08': 'data/T34UDG_20220417T095029_B08_10m.jp2',
   'SCL': 'data/T34UDG_20220417T095029_SCL_20m.jp2'}},
 'T34VDH': {'20220422T095031': {'B02': 'data/T34VDH_20220422T095031_B02_10m.jp2',
   'B03': 'data/T34VDH_20220422T095031_B03_10m.jp2',

In [7]:
s = 0
c = 4550

sj = 200

offsets = {
    'T34UDG': (30000+s, 0),
    'T34UEG': (30000+s, 10000),
    'T34UFG': (30000+s, 20000),
    'T34VDH': (20000+s, 0),
    'T34VDJ': (10000+s, 0),
    'T34VDK': (0+s, 0), # top left
    'T34VEH': (20000+s, 10000),
    'T34VEJ': (10000+s, 10000),
    'T34VEK': (0+s, 10000),
    'T34VFH': (20000+s, 20000),
    'T34VFJ': (10000+s, 20000),
    'T34VFK': (0+s, 20000),
    'T35ULB': (30000+sj, 30980-c),
    'T35UMB': (30000+sj, 40980-c),
    'T35UNB': (30000+sj, 50980-c),
    'T35VLC': (20000+sj, 30980-c),
    'T35VLD': (10000+sj, 30980-c),
    'T35VLE': (0+sj, 30980-c),
    'T35VMC': (20000+sj, 40980-c),
    'T35VMD': (10000+sj, 40980-c),
    'T35VME': (0+sj, 40980-c),
    'T35VNC': (20000+sj, 50980-c),
    'T35VND': (10000+sj, 50980-c),
    'T35VNE': (0+sj, 50980-c)
}


In [8]:
rgb_composite = np.dstack(
    (np.full((21680, 47410), 0.0), np.full((21680, 47410), 0.0), np.full((21680, 47410), 0.0))
)


In [9]:
cells = ["T34VFK", "T34VFJ", "T35VLE"]


In [10]:
dates = []
for c in cells:
    dates += list(files[c])
dates = sorted(list(set(dates)))
    

In [11]:
files = {k:v for k,v in files.items() if k in cells}


In [9]:
gain = 2

for d in dates:
    print(d)
    for ki, vi in files.items():
        if d in vi:
            print("matched", ki, d)
            fj = vi[d]
            try:
                try:
                    #print("reading SCL", fj["SCL"])
                    scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                except BaseException as e:
                    if os.path.isfile(fj["SCL"]):
                        print("REMOVED", fj["SCL"])
                        os.remove(fj["SCL"])
                    raise e
                
                try:
                    #print("reading red", fj["B04"])
                    c_red = np.clip(rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                except BaseException as e:
                    if os.path.isfile(fj["B04"]):
                        print("REMOVED", fj["B04"])
                        os.remove(fj["B04"])
                    raise e
                try:
                    #print("reading green", fj["B03"])
                    c_green = np.clip(rasterio.open(fj["B03"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)             
                except BaseException as e:
                    if os.path.isfile(fj["B03"]):
                        print("REMOVED", fj["B03"])
                        os.remove(fj["B03"])
                    raise e
                try:
                    #print("reading blue", fj["B02"])
                    c_blue = np.clip(rasterio.open(fj["B02"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)              
                except BaseException as e:
                    if os.path.isfile(fj["B02"]):
                        print("REMOVED", fj["B02"])
                        os.remove(fj["B02"])
                    raise e

                mask = ((scl != 0) & (scl != 1))
                #print(mask)
                #red[mask] = c_red[mask]
                #green[mask] = c_green[mask]
                #blue[mask] = c_blue[mask]
                #print("applying red")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 0][mask] = c_red[mask]
                #print("applying green")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 1][mask] = c_green[mask]
                #print("applying blue")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 2][mask] = c_green[mask]

            except BaseException as e:
                print(f"ERROR: {ki} {d} failed with {e}")
        else:
            print("no match", ki, d)
    print("saving composite")
    matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/notebooks/experiment/{d}.jpeg", rgb_composite)


20220419T094041
matched T35VLE 20220419T094041
matched T34VFK 20220419T094041
no match T34VFJ 20220419T094041
saving composite
20220421T093029
matched T35VLE 20220421T093029
no match T34VFK 20220421T093029
no match T34VFJ 20220421T093029
saving composite
20220430T100019
no match T35VLE 20220430T100019
no match T34VFK 20220430T100019
matched T34VFJ 20220430T100019
saving composite


In [ ]:
for d in dates:
    print(d)
    for ki, vi in files.items():
        if d in vi:
            print("matched", ki, d)
            fj = vi[d]
            try:
                try:
                    #print("reading SCL", fj["SCL"])
                    scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                except BaseException as e:
                    if os.path.isfile(fj["SCL"]):
                        print("REMOVED", fj["SCL"])
                        os.remove(fj["SCL"])
                    raise e
                
                try:
                    #print("reading red", fj["B04"])
                    c_b04 = rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)/10000
                except BaseException as e:
                    if os.path.isfile(fj["B04"]):
                        print("REMOVED", fj["B04"])
                        os.remove(fj["B04"])
                    raise e
                try:
                    c_b08 = rasterio.open(fj["B08"], driver="JP2OpenJPEG").read(1)/10000           
                except BaseException as e:
                    if os.path.isfile(fj["B08"]):
                        print("REMOVED", fj["B08"])
                        os.remove(fj["B08"])
                    raise e

                mask = ((scl != 0) & (scl != 1) & (scl != 3) & (scl != 7) & (scl != 8) & (scl != 9) & (scl != 10))
                ndvi = ((c_b08-c_b04)/(c_b08+c_b04))
                clipped = np.clip(ndvi, 0.0, 1.0)
                clipped[clipped < 0.4] = 0.0
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 1][mask] = clipped[mask]

            except BaseException as e:
                print(f"ERROR: {ki} {d} failed with {e}")
        else:
            print("no match", ki, d)
    print("saving composite")
    matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/notebooks/experiment/{d}.jpeg", rgb_composite)


20220419T094041
matched T35VLE 20220419T094041


/tmp/ipykernel_31365/2813762970.py:34: RuntimeWarning: invalid value encountered in divide
  ndvi = ((c_b08-c_b04)/(c_b08+c_b04))


matched T34VFK 20220419T094041
no match T34VFJ 20220419T094041
saving composite
20220421T093029
matched T35VLE 20220421T093029
no match T34VFK 20220421T093029
no match T34VFJ 20220421T093029
saving composite
20220430T100019
no match T35VLE 20220430T100019
no match T34VFK 20220430T100019
matched T34VFJ 20220430T100019
saving composite
